In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib editdistance
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer, LlamaForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd

from utils import similar_tag

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll
CUDA SETUP: CUDA runtime path found: C:\Users\panta\anaconda3\envs\nlp\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll...


c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: C:\Users\panta\anaconda3\envs\nlp did not contain ['cudart64_110.dll', 'cudart64_120.dll'] as expected! Searching further paths...
  warn(msg)
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\bitsandbytes\cuda_setup\main.py:152: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/usr/bin'), WindowsPath('C:/Users/panta/anaconda3/envs/nlp/Library/mingw-w64/bin')}
  warn(msg)


In [2]:
device = "cuda"
peft_model_id = r"ooferdoodles/text2tags-llama7b"
base_model = "decapoda-research/llama-7b-hf"
config = PeftConfig.from_pretrained(peft_model_id)
model = LlamaForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = LlamaTokenizer.from_pretrained(base_model)

model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16)
model.config

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\panta\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


LlamaConfig {
  "_name_or_path": "decapoda-research/llama-7b-hf",
  "architectures": [
    "LLaMAForCausalLM"
  ],
  "bos_token_id": 0,
  "eos_token_id": 1,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "max_sequence_length": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "pad_token_id": -1,
  "quantization_config": {
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_8bit": true
  },
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.29.0.dev0",
  "use_cache": true,
  "vocab_size": 32000
}

In [3]:
data = load_dataset("json", data_files=r"dataset/test_data.json")
tag_dict = similar_tag.load_dict()
data

Found cached dataset json (C:/Users/panta/.cache/huggingface/datasets/json/default-5cff6cb24ab66f93/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'tag_string'],
        num_rows: 150
    })
})

In [4]:
generation_config = GenerationConfig(
    temperature=0.7,
    top_p=0.75,
    top_k=40,
    num_beams=1,
    typical_p=1,
    do_sample=False,
    max_new_tokens=400,
    no_repeat_ngram_size=3,
    # pad_token_id=model.config.eos_token_id
    # truncation_length=2048,
    # min_length=0,
    # add_bos_token=True,
    # ban_eos_token=False,
    # skip_special_tokens=True,
    # stopping_strings=[],
    # penalty_alpha=0,
    # repetition_penalty=1.2,
    # encoder_repetition_penalty=1,
)

In [5]:
def pipe(data_point):
    # max_new_tokens = int(len(data_point['caption_string']) * max_token_scale)
    prompt = f"### Caption: {data_point['caption_string']}\n### Tags: "
    tokenized_prompt = tokenizer(prompt, return_tensors='pt').to(device)
    
    output_tokens = model.generate(
        input_ids=tokenized_prompt['input_ids'],
        generation_config=generation_config,
    )
    preds = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    pred_list = [x.strip() for x in preds.split('### Tags:')[-1].split(",")]
    corrected_tags = similar_tag.correct_tags(pred_list, tag_dict)
    data_point['tags'] = data_point['tag_string'].split(', ')
    data_point['pred_tags'] = corrected_tags
    return data_point

In [6]:
processed_data = data.map(pipe)
processed_data

Parameter 'function'=<function pipe at 0x0000026CACD330A0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 8.00 GiB total capacity; 7.18 GiB already allocated; 0 bytes free; 7.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def evaluate_accuracy(data_point):
    correct_count = len(set(data_point['tags']).intersection(data_point['pred_tags']))
    # incorrect_count = len(data_point['tags']) - correct_count
    data_point['accuracy'] = correct_count / len(data_point['tags']) * 100
    return data_point

In [ ]:
evaluated_data = processed_data.map(evaluate_accuracy)
evaluated_data

In [ ]:
df = evaluated_data['train'].to_pandas()
df

In [ ]:
df['accuracy'].mean()